# Common Crawl → Financial Features Pipeline

## Context
This notebook directly addresses the Numerai job description, which mentions "processing Common Crawl data" as a core responsibility. Common Crawl is the largest publicly available web crawl dataset — petabytes of HTML from billions of web pages.

The challenge: **extract financially relevant text from this haystack, map it to stock tickers, and produce clean features — all while maintaining strict point-in-time correctness.**

## My Infrastructure Experience
- **NDIF/NNsight** (ICLR 2025): Built Ray GCS Service backend with AWS object storage and VLLM for inference. Scaled to handle thousands of concurrent model probing requests.
- **NeuroData Lab**: Optimized a diffusion MRI pipeline with Kubernetes, Docker, and AWS Batch. Halved runtime, cut cloud costs 40%.
- **Creyon Bio**: Scaled data processing pipelines for protein sequence analysis.

## Pipeline Architecture
```
Common Crawl (WARC files, ~3TB/month)
    → Filter: financial content classifier
    → Parse: extract clean text from HTML
    → Entity Link: map text → stock tickers
    → Feature Extract: sentiment, embeddings, linguistic features (NB01-07)
    → Temporal Aggregate: daily text → weekly features (match Numerai eras)
    → Output: per-ticker feature vectors for Numerai Signals
```

In [ ]:
import re
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
from collections import defaultdict, Counter
from urllib.parse import urlparse

## 1. Understanding Common Crawl Data

Common Crawl stores web pages in WARC (Web ARChive) format. Each monthly crawl contains:
- ~3 billion web pages
- ~200-400 TB compressed
- Available for free on S3: `s3://commoncrawl/`

Each WARC record contains:
- URL of the crawled page
- HTTP response headers
- HTML content
- Crawl timestamp (critical for point-in-time correctness)

In [ ]:
# Simulated Common Crawl WARC records
# In production: use warcio library to read actual WARC files from S3
# Example: aws s3 ls s3://commoncrawl/crawl-data/CC-MAIN-2024-10/segments/

# TODO: implement
...

## 2. Step 1: Filter for Financial Content
Most of Common Crawl is irrelevant (recipes, sports, etc). We need a fast filter.

In [ ]:
# Financial content filtering strategy:
# Level 1: URL-based filtering (fast, no parsing needed)
# Level 2: Keyword-based filtering (requires text extraction)
# Level 3: ML classifier (most accurate, most expensive)
def is_financial_url(url):
    """Level 1: Fast URL-based filter."""
    ...

def is_financial_content(text, threshold=3):
    """Level 2: Keyword-based content filter."""
    ...

def filter_pipeline(record):
    """Multi-level financial content filter."""
    ...


## 3. Step 2: Parse HTML → Clean Text

In [ ]:
def extract_text(html):
    """Extract clean text from HTML. In production, use BeautifulSoup or trafilatura."""
    ...

def extract_title(html):
    """Extract title from HTML."""
    ...


## 4. Step 3: Entity Linking (Text → Stock Tickers)
Map company mentions to stock tickers. This is a critical and non-trivial step.

In [ ]:
# Entity linking: company name → stock ticker
# In production: use a comprehensive database + fuzzy matching + NER
def link_entities(text):
    """Map company mentions in text to stock tickers."""
    ...


## 5. Step 4: Point-in-Time Correctness (CRITICAL)
This is the most important engineering constraint. Features must only use information available at prediction time.

In [ ]:
# Point-in-time correctness demonstration
# 
# KEY RULE: Use the CRAWL TIMESTAMP, not the publication date
# Why? The crawl timestamp is when the information was definitely available
# Publication dates can be:
#   - Missing or unreliable
#   - Backdated
#   - Different from when the content was actually accessible
#
# Numerai eras are weekly (Friday close). Features for era T must only use
# data crawled BEFORE era T's prediction deadline.
def assign_to_era(crawl_timestamp, era_schedule):
    """Assign a document to the latest era it's available for."""
    ...


## 6. Step 5: Temporal Aggregation
Aggregate text features per ticker per era, with exponential decay weighting.

In [ ]:
# Temporal aggregation with exponential decay
# Recent documents are weighted more heavily than older ones
def exponential_decay_weight(days_old, half_life=7):
    """Weight that halves every `half_life` days."""
    ...


## 7. Scaling Architecture

In [ ]:
# Scaling calculations for Common Crawl processing

# TODO: implement
...

## Discussion & Interview Talking Points

### Why I Can Do This
- **NDIF** (ICLR 2025): I built a distributed inference platform using Ray, AWS, and VLLM that handles thousands of concurrent probing requests. Common Crawl processing is a similar distributed data pipeline.
- **NeuroData MRI Pipeline**: Optimized a Kubernetes + AWS Batch pipeline for processing neuroimaging data. Halved runtime, cut cloud costs 40%. Same principles apply: parallelism, fault tolerance, cost optimization.
- **Scale mindset**: I've worked with petabyte-scale data (neuroimaging), distributed training, and production ML systems.

### Key Engineering Challenges
1. **Deduplication**: Common Crawl has many duplicate pages across monthly crawls. Need MinHash/SimHash dedup.
2. **Language filtering**: Most of CC is English, but need to handle multilingual content for global stocks.
3. **Rate limiting**: If supplementing CC with live API calls, respect rate limits.
4. **Storage**: Processed features for 5,000 stocks x 52 weeks x N features = manageable, but raw text is huge.
5. **Monitoring**: Need observability for a pipeline this complex (logging, alerting, data quality checks).

### Technology Stack I'd Use
| Component | Technology | Why |
|-----------|-----------|-----|
| Orchestration | Ray / Spark | Distributed processing, fault tolerance |
| Storage | S3 + Parquet | Columnar format, cheap storage |
| Compute | AWS Batch / EC2 | Spot instances for cost savings |
| GPU inference | VLLM / TGI | Batched inference for embeddings |
| Monitoring | W&B / MLflow | Already experienced with both |
| Scheduling | Airflow / Prefect | Weekly pipeline runs |

### Extensions (TODO)
- [ ] Download and process a real CC WARC segment (warcio + boto3)
- [ ] Implement MinHash deduplication for cross-crawl dedup
- [ ] Build a financial content classifier (fine-tune BERT on financial vs non-financial)
- [ ] Add comprehensive entity linking with fuzzy matching (fuzzywuzzy + company database)
- [ ] Estimate actual AWS costs for monthly processing
- [ ] Set up an Airflow DAG for weekly pipeline execution